In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
import nltk
import statistics
import numpy as np
import pickle as pkl
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K 
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#from textblob import Word
import nltk
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>


False

In [3]:
X=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
test=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [4]:
X1_train = X.loc[:int(len(X.index)*0.8)]
y_train = X1_train[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]
X1_val = X.loc[int(len(X.index)*0.8)+1:int(len(X.index)*0.9)]
y_val = X1_val[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']]
X1_test = X.loc[int(len(X.index)*0.9)+1:]
y_test = X1_test[['syntax','cohesion','vocabulary','phraseology','grammar','conventions']] 

#X1_train is only for extracting out y for each set.

In [5]:
IP=14

y1_train=y_train["syntax"]
y2_train=y_train["cohesion"]
y3_train=y_train["vocabulary"]
y4_train=y_train["phraseology"]
y5_train=y_train["grammar"]
y6_train=y_train["conventions"]

y1_val=y_val["syntax"]
y2_val=y_val["cohesion"]
y3_val=y_val["vocabulary"]
y4_val=y_val["phraseology"]
y5_val=y_val["grammar"]
y6_val=y_val["conventions"]

y1_test=y_test["syntax"]
y2_test=y_test["cohesion"]
y3_test=y_test["vocabulary"]
y4_test=y_test["phraseology"]
y5_test=y_test["grammar"]
y6_test=y_test["conventions"]


y1_train = np.asarray(y1_train)
y1_val = np.asarray(y1_val)  

y2_train = np.asarray(y2_train)
y2_val = np.asarray(y2_val)  

y3_train = np.asarray(y3_train)
y3_val = np.asarray(y3_val)  

y4_train = np.asarray(y4_train)
y4_val = np.asarray(y4_val)  

y5_train = np.asarray(y5_train)
y5_val = np.asarray(y5_val)  

y6_train = np.asarray(y6_train)
y6_val = np.asarray(y6_val)  


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
def tokenize_sentences(data):
    sent_token = nltk.tokenize.sent_tokenize(data)
    return sent_token, len(sent_token)

# calls the tokenize_sentences method and returns the total number of sentences
def sent_count(data):
    return tokenize_sentences(data)[1]

In [8]:
def tokenize_words(data, punc=False, remove_stop_words=False, lower=True):
    if lower:
        word_tokens = nltk.tokenize.word_tokenize(data.lower())
    else:
        word_tokens = nltk.tokenize.word_tokenize(data)
    if punc:
        word_tokens = [word for word in word_tokens if word.isalnum()]
    if remove_stop_words:
        stop_words = get_stop_words()
        word_tokens = [word for word in word_tokens if word not in stop_words]
        
    return word_tokens, list(set(word_tokens)), len(word_tokens)

# calls the tokenize_words method and returns the total number of tokens
def word_count(data):
    return tokenize_words(data)[2]

In [9]:
def token_frequency(words):
    frequency_of_tokens = nltk.FreqDist(words)
    return frequency_of_tokens

In [10]:
def token_length(data):
    words = tokenize_words(data)[0]
    len_of_tokens = {}
    for word in words:
        len_of_tokens[word] = len(word)
    return len_of_tokens
# calls the token_length method and returns the average of token lengths
def avg_length(words):
    return statistics.mean(token_length(words).values())

In [11]:
def sentence_length(sent):
    len_of_sent = []
    for s in sent:
        len_of_sent.append(tokenize_words(s)[2])
    return len_of_sent, statistics.mean(len_of_sent)
def avg_word_sentence(data):
        return sentence_length(tokenize_sentences(data)[0])[1]

In [12]:
def char_count(data):
    return len(data.lower().replace(' ',''))

In [13]:
def pos_tags(data):
    sent = tokenize_sentences(data)[0]
    
    noun_count = 0
    adj_count = 0
    verb_count = 0
    adv_count = 0
    adp_count=0
    conj_count = 0
    det_count = 0
    prt_count = 0
    pron_count = 0
    punct_count=0
    
    
    for s in sent:
        tags = nltk.pos_tag(tokenize_words(s)[0])
        #print(tags)
        for tag in tags:
            if tag[1][0] == 'N':
                noun_count += 1
            elif tag[1][0] == 'J':
                adj_count += 1
            elif tag[1][0] == 'V':
                verb_count += 1
            elif tag[1][0] == 'R':
                adv_count += 1
            elif tag[1] == 'IN':
                adp_count += 1
            elif tag[1] == 'CC':
                conj_count += 1
            elif tag[1] == 'DT':
                det_count += 1
            elif tag[1] == 'PRT':
                prt_count += 1
            elif tag[1] == 'PRP' or tag[1] == 'PRP$' :
                pron_count += 1
            elif tag[1] == '.':
                punct_count += 1
                
    return noun_count,adj_count,verb_count,adv_count , adp_count , conj_count, det_count , pron_count , punct_count


pos_tags(" new good on at relly already and or the some year home at on he their his say told ! ")

(3, 2, 1, 2, 4, 2, 2, 3, 1)

In [14]:
X['num_words'] = X['full_text'].apply(word_count)
test['num_words'] = test['full_text'].apply(word_count)
test.head()  


,text_id,full_text,num_words
0,0000C359D63E,when a person has no experience on a job their...,874
1,000BAD50D026,Do you think students would benefit from being...,421
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",484


In [15]:
X['num_sents'] = X['full_text'].apply(sent_count)
test['num_sents'] = test['full_text'].apply(sent_count)
test.head()  

,text_id,full_text,num_words,num_sents
0,0000C359D63E,when a person has no experience on a job their...,874,26
1,000BAD50D026,Do you think students would benefit from being...,421,17
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",484,17


In [16]:
X['num_sents'] = X['full_text'].apply(sent_count)
test['num_sents'] = test['full_text'].apply(sent_count)
test.head()  

,text_id,full_text,num_words,num_sents
0,0000C359D63E,when a person has no experience on a job their...,874,26
1,000BAD50D026,Do you think students would benefit from being...,421,17
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",484,17


In [17]:
X['avg_word_sent'] = X['full_text'].apply(avg_word_sentence)
test['avg_word_sent'] = test['full_text'].apply(avg_word_sentence)
test.head()  

,text_id,full_text,num_words,num_sents,avg_word_sent
0,0000C359D63E,when a person has no experience on a job their...,874,26,33.615385
1,000BAD50D026,Do you think students would benefit from being...,421,17,24.764706
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",484,17,28.470588


In [18]:
X['noun_count'], X['adj_count'], X['verb_count'], X['adv_count'], X['adp_count'],X['conj_count'], X['det_count'], X['pron_count'], X['punct_count']  = zip(*X['full_text'].map(pos_tags))
              
test['noun_count'], test['adj_count'], test['verb_count'], test['adv_count'], test['adp_count'],test['conj_count'], test['det_count'], test['pron_count'], test['punct_count']  = zip(*test['full_text'].map(pos_tags))
                  

In [19]:
X.head()
test.head()
print(X.shape)
test.shape

(3911, 20)


(3, 14)

In [20]:
features = X.iloc[:,8:]
final_test = test.iloc[:,2:]
# print(features.head)
features_array = np.asarray(features)
features_array_train = features_array[:3129]
features_array_val = features_array[3129:3129+391]
features_array_test = features_array[3129+391:]
print(features_array_test.shape)
print(features_array_val.shape)
print(features_array_train.shape)
final_test.shape


(391, 12)
(391, 12)
(3129, 12)


(3, 12)

In [21]:
lr1 = LinearRegression()
lr2 = LinearRegression()
lr3 = LinearRegression()
lr4 = LinearRegression()
lr5 = LinearRegression()
lr6 = LinearRegression()

lr1.fit(features_array_train, y1_train)
lr2.fit(features_array_train, y2_train)
lr3.fit(features_array_train, y3_train)
lr4.fit(features_array_train, y4_train)
lr5.fit(features_array_train, y5_train)
lr6.fit(features_array_train, y6_train)



y1_pred = lr1.predict(features_array_test)
y2_pred = lr2.predict(features_array_test)
y3_pred = lr3.predict(features_array_test)
y4_pred = lr4.predict(features_array_test)
y5_pred = lr5.predict(features_array_test)
y6_pred = lr6.predict(features_array_test)

y1_pred = [round(r,1) for r in y1_pred]
y2_pred = [round(r,1) for r in y2_pred]
y3_pred = [round(r,1) for r in y3_pred]
y4_pred = [round(r,1) for r in y4_pred]
y5_pred = [round(r,1) for r in y5_pred]
y6_pred = [round(r,1) for r in y6_pred]


values1 = mean_squared_error(y1_pred, y1_test)
values2 = mean_squared_error(y2_pred, y2_test)
values3 = mean_squared_error(y3_pred, y3_test)
values4 = mean_squared_error(y4_pred, y4_test)
values5 = mean_squared_error(y5_pred, y5_test)
values6 = mean_squared_error(y6_pred, y6_test)


print(values1,values2,values3,values4,values5,values6)  


0.33324808184143223 0.3563171355498721 0.2715089514066496 0.34687979539641944 0.4433759590792839 0.3734271099744245


In [22]:
print(final_test.shape)
final_test

(3, 12)


,num_words,num_sents,avg_word_sent,noun_count,adj_count,verb_count,adv_count,adp_count,conj_count,det_count,pron_count,punct_count
0,874,26,33.615385,154,40,189,44,90,22,97,111,26
1,421,17,24.764706,88,14,87,18,54,19,19,59,17
2,484,17,28.470588,65,30,107,27,54,14,25,73,17


In [23]:
y1_pred = lr1.predict(final_test)
y2_pred = lr2.predict(final_test)
y3_pred = lr3.predict(final_test)
y4_pred = lr4.predict(final_test)
y5_pred = lr5.predict(final_test)
y6_pred = lr6.predict(final_test)


test1 = [round(r,1) for r in y1_pred]
test2 = [round(r,1) for r in y2_pred]
test3 = [round(r,1) for r in y3_pred]
test4 = [round(r,1) for r in y4_pred]
test5 = [round(r,1) for r in y5_pred]
test6 = [round(r,1) for r in y6_pred]


print(test1)
print(test2)
print(test3)
print(test4)
print(test5)
print(test6)


[3.0, 2.9, 3.1]
[3.0, 3.0, 3.1]
[3.2, 3.1, 3.2]
[3.0, 3.0, 3.1]
[2.7, 2.9, 3.0]
[2.9, 3.0, 3.1]


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegr

In [24]:
t1=pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")

In [25]:
final=pd.DataFrame()

final['text_id']=t1["text_id"]
final['syntax']=test1
final['cohesion']=test2
final['vocabulary']=test3
final['phraseology']=test4
final['grammar']=test5
final['conventions']=test6  

In [26]:
final.to_csv('submission.csv', index=False)

In [27]:
final

,text_id,syntax,cohesion,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.2,3.0,2.7,2.9
1,000BAD50D026,2.9,3.0,3.1,3.0,2.9,3.0
2,00367BB2546B,3.1,3.1,3.2,3.1,3.0,3.1
